#### What Are AI Agents?
AI agents are specialized programs or models designed to perform tasks autonomously using artificial intelligence techniques, often mimicking human decision-making, reasoning, and learning. They interact with users or systems, learn from data, adapt to new information, and execute specific functions within a defined scope, like customer support, process automation, or complex data analysis.

In the real world, tasks rarely have single-step solutions. Instead, they typically involve a series of interconnected and standalone steps to be carried out. For example, for a question like –

“Which coffee had the highest sales in our Manhattan based store?” might have a single step answer.

However, for a question like –

“Which 3 coffee types would be liked by our customer Emily who works at Google, NYC office? She prefers low calories coffee and likes Lattes more than Cappuccinos. Also could you send a mail to her with a promotional campaign for these 3 coffee types mentioning the nearest location of our store to her office where she can grab these?”

A single LLM would not be able to handle such a complex query by itself and this is where the need for an AI agent consisting of multiple LLMs arises.

For handling such complex tasks, instead of prompting a single LLM, multiple LLMs can be combined together acting as AI agents to break the complex task into multiple independent tasks.

#### Step-by-Step Python Implementation
Let us consider a use case in which a coffee chain like Starbucks wants to build an AI agent for drafting and mailing personalized promotional campaigns recommending 3 types of coffee for their customers based on their coffee preferences. The promotional campaign should also include the location of the coffee store which is nearest to the customer’s location where the customer can easily grab these coffees.

We will be using the Starbucks Data here which has information on different types of Starbucks Coffee along with their nutritional information. 

In [2]:
# !pip install llama-index-core
# !pip install llama-index-readers-file
# !pip install llama-index-embeddings-openai
# !pip install llama-index-llms-llama-api
# !pip install 'crewai[tools]'
# !pip install llama-index-llms-langchain
# pip install transformers sentence-transformers langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.1 MB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 27.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.12
    Uninstalling pydantic-1.10.12:
      Successfully uninstalled pydantic-1.10.12
  Attempting 

In [5]:
import pandas as pd
import numpy as np
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import LlamaIndexTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI
from langchain_openai import ChatOpenAI

from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import CSVLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import warnings
warnings.filterwarnings('ignore')

In [3]:
def read_data(path_to_csv_file):
    '''
    Reads csv files from specified paths

    Parameters:
            path to csv file locations
    Returns:
            dataframes of imported csv files
    '''
    # Read CSV files
    df = pd.read_csv(path_to_csv_file)
    
    return df

# call the function to import train data - update with relevant paths
data = read_data('/Users/mncedisimncwabe/Downloads/starbucks.csv')

In [3]:
data.head()

,Beverage_category,Beverage,Beverage_prep,Calories,Total Fat (g),Trans Fat (g),Saturated Fat (g),Sodium (mg),Total Carbohydrates (g),Cholesterol (mg),Dietary Fibre (g),Sugars (g),Protein (g),Vitamin A (% DV),Vitamin C (% DV),Calcium (% DV),Iron (% DV),Caffeine (mg)
0,Coffee,Brewed Coffee,Short,3,0.1,0.0,0.0,0,5,0,0,0,0.3,0%,0%,0%,0%,175
1,Coffee,Brewed Coffee,Tall,4,0.1,0.0,0.0,0,10,0,0,0,0.5,0%,0%,0%,0%,260
2,Coffee,Brewed Coffee,Grande,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,0%,0%,330
3,Coffee,Brewed Coffee,Venti,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,2%,0%,410
4,Classic Espresso Drinks,Caffè Latte,Short Nonfat Milk,70,0.1,0.1,0.0,5,75,10,0,9,6.0,10%,0%,20%,0%,75


In [4]:
data.shape

(242, 18)

In [5]:
openai_api_key = ''
os.environ['OPENAI_API_KEY']=openai_api_key

In [8]:
# Load Data using LlamaIndex’s SimpleDirectoryReader
reader = SimpleDirectoryReader(input_files=['/Users/mncedisimncwabe/Downloads/starbucks.csv'])
docs = reader.load_data()
docs

[Document(id_='7982dd79-2ea6-4489-8a7b-dca5dd92b055', embedding=None, metadata={'file_path': '/Users/mncedisimncwabe/Downloads/starbucks.csv', 'file_name': 'starbucks.csv', 'file_type': 'text/csv', 'file_size': 26898, 'creation_date': '2024-12-07', 'last_modified_date': '2024-12-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Coffee, Brewed Coffee, Short, 3, 0.1, 0.0, 0.0, 0, 5, 0, 0, 0, 0.3, 0%, 0%, 0%, 0%, 175\nCoffee, Brewed Coffee, Tall, 4, 0.1, 0.0, 0.0, 0, 10, 0, 0, 0, 0.5, 0%, 0%, 0%, 0%, 260\nCoffee, Brewed Coffee, Grande, 5, 0.1, 0.0, 0.0, 0, 10, 0, 0, 0, 1.0, 0%, 0%, 0%, 0%, 330\nCoffee, Brewed Coffee, Venti, 5, 0.1, 0.0, 0.0, 0, 10, 0, 0, 0, 1.0, 0%, 0%, 2%, 0%, 410\nClassic Es

In [9]:
# Create Query Tool For Interacting With the CSV Data

# We have used gpt-4o model here as the LLM. Other OpenAI models can also be used
llm = ChatOpenAI(temperature=0, model="gpt-4o", max_tokens=1000)

# creates a VectorStoreIndex from a list of documents (docs)
index = VectorStoreIndex.from_documents(docs)

# The vector store is transformed into a query engine. 
# Setting similarity_top_k=5 limits the results to the top 5 documents that are most similar to the query, 
# llm specifies that the LLM should be used to process and refine the query results
query_engine = index.as_query_engine(similarity_top_k=5, llm=llm)
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Coffee Promo Campaign",
    description="Use this tool to lookup the Starbucks Coffee Dataset",
)

ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******